In [1]:
import numpy as np
import importlib
# import os

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
from FactoryCalculator.Entities import Belt,Miner,Module,Inserter,Factory,Chest

In [135]:
sMod1 = Module('speed',1)
sMod2 = Module('speed',2)
sMod3 = Module('speed',3)
miner = Miner('electric',material='copper',lvl=3)
miner.add_modules([sMod1,sMod2])
miner.mining_base_speed
miner.mining_speed

miner_ouput = miner.get_output()

b = Belt('yellow')
print(b.get_content())
b.load(miner_ouput)
print(b.get_content())

I = Inserter('regular',lvl=1)
I.link_source(b,'copper')
I.get_output_max('copper',output_type='factory')

chest = Chest()

f = Factory('reg_fact',{'copper':1},0.5,{'copperCable':2})

f.set_factory_io([I],[chest])
f.produce()
print(b.get_content())

{}
{'copper': 0.9359999999999999}
-----
copper requires 2.0 input but only 0.9359999999999999 is available. Scaling production by factor 0.46799999999999997
-----
Factory report for factory reg_fact, producing ['copperCable']
I/O scaled by factor of 0.46799999999999997
-----
{}


#### Other stuff to implement

Modules (these can be tagged in the relevant classes)

Energy consumption

Pollution

Power plants